# Principal Component Analysis (PCA)

In this exercise, we will use the same dataset as for the linear regression calculation. 

Recall that based on the correlation matrix, we selected the variables RM, LSTAT and the explanatory variable MEDV in the model.

The developed model had an R2 score of approximately 0.65 on the training and test data.

Using PCA, we will try to achieve a better result, i.e. to create a model that will predict better.

## Data loading and analysis

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
data = pd.read_csv ("..\\dataset\\HousingData.csv")

In [ ]:
data=data.dropna()

In [ ]:
data.head()

In [ ]:
data.describe()

The scales of the variables are very different from each other, so we will have to rescale our data to improve its quality because we cannot apply PCA or linear regression to these data.

## Linear model of all variables without adjustments
Create a control linear model with all variables. 

The goal is to have a sample model to compare with the refinements.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

We divide the data into training and testing

In [ ]:
X = np.array(data.drop('MEDV',axis=1))
Y = np.array(data['MEDV'])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

Creating a linear model

In [ ]:
lr = LinearRegression()
lr.fit(X_train, Y_train)

Validation of the model on test data

In [ ]:
Y_pred = lr.predict(X_test)
r2 = r2_score(Y_test, Y_pred)
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

Validation of the model on training data

In [ ]:
Y_pred = lr.predict(X_train)
r2 = r2_score(Y_train, Y_pred)
rmse = np.sqrt(mean_squared_error(Y_train, Y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

## Correlation
Again we perform a correlation analysis and look for linear dependencies between the variables.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
sns.heatmap(data.corr(),annot=True)

The **correlation matrix** contains Pearson correlation coefficients between all pairs of variables. The values range from -1 to 1.
* 1 → strong positive linear correlation
* -1 → strong negative linear correlation
* 0 → no linear correlation

**Multicollinearity** occurs when predictor variables are strongly correlated with each other (e.g. DIS with INUDS, INOX, AGE).

Problem:
* Estimates of regression coefficients are unstable and sensitive to small changes in the data.
* Interpretation of individual coefficients becomes meaningless because it is impossible to isolate the effect of one variable while holding the others fixed if they are correlated with each other.
* Increases the standard error of the coefficients, which reduces statistical significance.


The last row is important, as it shows us the linear correlation between the explanatory variables and the explained variable MEDV. Our target variable seems to be highly correlated with LSTAT and RM, which makes sense since these two factors are very important for house pricing.  There is also a lot of multicollinearity.

The usual interpretation of the regression coefficient is that it provides an estimate of the effect of a unit change in the independent variable on the dependent variable while holding the other variables constant. In the case of multicollinearity, however, we cannot say this. If X1 is strongly correlated with another independent variable, X2, in a given data set, then we have a set of observations for which X1 and X2 have some linear stochastic relationship. Thus, we cannot ensure when the variable X1 changes, X2 remains constant.


## Variance Inflation Factor (VIF) 

The **VIF** detects multicollinearity in regression analysis. 

Its presence can negatively affect the regression results. The VIF estimates how much the variance of the regression coefficient is inflated due to multicollinearity in the model.

VIF=1/(1-R^2)

Where R^2 is the coefficient of determination. 

Simply put, it is the proportion of the variance of the independent variable that is explained by the dependent variable. 

So we run a linear regression using each variable as the target and the others as predictors and calculate R^2 and then calculate VIF for them.

If VIF < 4, the variable can be used, otherwise we have to find a way to remove the collinearity of these variables.

In [ ]:
vifdf = []
for i in data.columns:
    X = np.array(data.drop(i,axis=1))
    y = np.array(data[i])
    lr = LinearRegression()
    lr.fit(X,y)
    y_pred = lr.predict(X)
    r2 = r2_score(y,y_pred)
    vif = 1/(1-r2)
    vifdf.append((i,vif))

vifdf = pd.DataFrame(vifdf,columns=['Features','Variance Inflation Factor'])
vifdf.sort_values(by='Variance Inflation Factor')

We see that almost half of the variables have a VIF value greater than or close to 4. TAX and RAD have a VIF almost twice as high as our threshold.

Thus, it will be appropriate to resolve multicollinearity. This can be done in several ways:
* Remove correlated variables → select only one of the pair of highly correlated variables.
* Principal Component Analysis (PCA) → transform the predictors into uncorrelated components.
* Regularization (Ridge, Lasso) → suppresses the effect of collinearity and stabilizes the model.

We will look at PCA.

## Data standardization
The first step is to standardize the data, so that all variables have a mean of around 0. Then their effect on the output variable will be similar.

Let's look at the data before standardization.

In [ ]:
pos = 1
fig = plt.figure(figsize=(8, 12))
for i in data.columns:
    ax = fig.add_subplot(7,2, pos)
    pos = pos + 1
    sns.histplot(data[i],ax=ax, kde=True)

Perform z-standardization using the rescale function.

In [ ]:
def rescale(X):
    mean = X.mean()
    std = X.std()
    scaled_X = [(i - mean)/std for i in X]
    return pd.Series(scaled_X)

Create a new standardized dataset data_std.

In [ ]:
data_std = pd.DataFrame(columns=data.columns)
for i in data.columns:
    data_std[i] = rescale(data[i])

To check, let's write down the basic statistics.

In [ ]:
data_std.describe()

Display the distribution of values with an estimate of the distribution function.

The shape of the distribution of the new variables is the same as for the original variables. Only their mean value is now 0.

In [ ]:
pos = 1
fig = plt.figure(figsize=(8,12))
for i in data_std.columns:
    ax = fig.add_subplot(7,2, pos)
    pos = pos + 1
    sns.histplot(data_std[i],ax=ax, kde=True)

Let's look at the correlation of the standardized data. It remains the same.

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
sns.heatmap(data_std.corr(),annot=True)

## PCA

### Idea
* If we have a lot of correlated variables, there is hidden redundancy in the data.
* PCA removes this redundancy by converting the original variables to new, uncorrelated variables = principal components.
* These components are linear combinations of the original variables.


### How PCA works (intuition)
* Finds the direction with the largest variance of the data (1st principal component).
* Finds a second direction with the largest variance but orthogonal to the first (2nd principal component).
* Continues until all dimensions are exhausted.
* Result:
    * Principal components are uncorrelated.
    * The first components explain most of the variability in the data.

### What PCA is used for
* Removing multicollinearity → components are orthogonal → no collinearity.
* Dimensionality reduction → keep only the first few components that explain e.g. 90-95% of the variability.
* Visualization → complex data from many variables can be plotted in 2D/3D space.

PCA is sensitive to the scale of the variables. Therefore, standardization is usually done before applying PCA.

We will not write PCA by hand, but use its implementation from the library.


In [ ]:
from sklearn.decomposition import PCA

The number of PCA components will be 13 as well as input parameters.

We have to remove the output MEDV from the input to the PCA.

In [ ]:
pca = PCA(n_components=13)
X = data_std.drop('MEDV',axis=1)
X_pca = pca.fit_transform(X)

Now we will create a new data with principal components as input variables and MEDV as output variable.

In [ ]:
data_std_pca = pd.DataFrame(X_pca,columns=['PCA1','PCA2','PCA3','PCA4','PCA5','PCA6','PCA7','PCA8','PCA9','PCA10','PCA11','PCA12','PCA13'])
data_std_pca['MEDV'] = data_std['MEDV']

PCA was intended to reduce multicollinearity. Let's check it out.

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
sns.heatmap(data_std_pca.corr(),annot=True)

The correlation matrix shows that the PCA components are not dependent on each other.

MEDV is linearly dependent on the first 3 PCA variables. Then the linear dependence decreases greatly.

The distribution functions of the PCA variables are different from the original ones.

In [ ]:
pos = 1
fig = plt.figure(figsize=(12,16))
for i in data_std_pca.columns:
    ax = fig.add_subplot(7,2, pos)
    pos = pos + 1
    sns.histplot(data_std_pca[i],ax=ax, kde=True)

## Linear model of all PCA variables 
We again divide the data into training and test data.

In [ ]:
X = np.array(data_std_pca.drop('MEDV',axis=1))
Y = np.array(data_std_pca['MEDV'])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

We create a linear model.

In [ ]:
lr = LinearRegression()
lr.fit(X_train, Y_train)

Model validation for training data

In [ ]:
Y_pred = lr.predict(X_train)
r2 = r2_score(Y_train, Y_pred)
rmse = np.sqrt(mean_squared_error(Y_train, Y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

Model validation for test data

In [ ]:
Y_pred = lr.predict(X_test)
r2 = r2_score(Y_test, Y_pred)
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

The resulting model from the PCA variables is slightly better than the original linear model from the original variables.

## Linear model of 6 PCA variables
PCA can also be used for dimensionality reduction. 

So we create a model that has only 6 variables instead of 13 input variables.

In [ ]:
lr = LinearRegression()
lr.fit(X_train[:,0:6], Y_train)

Model validation on training data.

In [ ]:
Y_pred = lr.predict(X_train[:,0:6])
r2 = r2_score(Y_train, Y_pred)
rmse = np.sqrt(mean_squared_error(Y_train, Y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

Model validation on test data

In [ ]:
Y_pred = lr.predict(X_test[:,0:6])
r2 = r2_score(Y_test, Y_pred)
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))

print(f"R2 score: {r2}")
print(f"RMSE: {rmse}")

As expected, the precision of the reduced model is slightly lower. On the other hand, the model is smaller.